<a href="https://colab.research.google.com/github/Rjritik12/AI-camera/blob/main/parking_violation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install opencv-python numpy torch torchvision

In [22]:
!pip install ultralytics
import torch
from ultralytics import YOLO  # Import the YOLO class

# Try using the 'main' branch
model = YOLO('/content/best (1).pt')  # Make sure to use quotes around the model path

In [25]:
import cv2
cap = cv2.VideoCapture('video.mp4')  # or 'image.jpg'

In [14]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    for detection in results.xyxy:
        # Get bounding box coordinates
        x1, y1, x2, y2 = detection[:4].int().tolist()
        # Get class ID and confidence
        class_id = detection[5].int().item()
        confidence = detection[4].float().item()
        # Filter out low-confidence detections
        if confidence > 0.5:
            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_id} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [15]:
import time

# Define ROI coordinates
roi_x1, roi_y1, roi_x2, roi_y2 = 100, 100, 300, 300

# Initialize timer and violation log
timer = {}
viol_log = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    for detection in results.xyxy:
        # Get bounding box coordinates
        x1, y1, x2, y2 = detection[:4].int().tolist()
        # Check if vehicle is in ROI
        if roi_x1 < x1 < roi_x2 and roi_y1 < y1 < roi_y2:
            # Get vehicle ID and class
            vehicle_id = detection[5].int().item()
            class_name = detection[6].int().item()
            # Check if vehicle has been in ROI for 10 seconds
            if vehicle_id in timer and time.time() - timer[vehicle_id] > 10:
                # Log violation and save image
                viol_log.append((class_name, vehicle_id, time.time()))
                cropped = frame[y1:y2, x1:x2]
                cv2.imwrite(f'outputs/violations/{class_name}_{vehicle_id}.jpg', cropped)
                print(f'Violation detected: {class_name} {vehicle_id}')
            else:
                # Start timer for vehicle
                timer[vehicle_id] = time.time()

In [16]:
import time

# Define ROI coordinates for red light
red_light_roi_x1, red_light_roi_y1, red_light_roi_x2, red_light_roi_y2 = 400, 400, 600, 600

# Initialize timer and violation log
timer = {}
viol_log = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    for detection in results.xyxy:
        # Get bounding box coordinates
        x1, y1, x2, y2 = detection[:4].int().tolist()
        # Check if vehicle is in ROI
        if red_light_roi_x1 < x1 < red_light_roi_x2 and red_light_roi_y1 < y1 < red_light_roi_y2:
            # Get vehicle ID and class
            vehicle_id = detection[5].int().item()
            class_name = detection[6].int().item()
            # Check if vehicle has crossed the red light
            if is_red_light(frame):
                # Log violation and save image
                viol_log.append((class_name, vehicle_id, time.time()))
                cropped = frame[y1:y2, x1:x2]
                cv2.imwrite(f'outputs/violations/{class_name}_{vehicle_id}.jpg', cropped)
                print(f'Red light crossing detected: {class_name} {vehicle_id}')

def is_red_light(frame):
    # Implement your red light detection logic here
    # For example, you can use color thresholding to detect the red color
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    return np.any(mask)

In [17]:
import numpy as np

# Initialize tracker
tracker = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    for detection in results.xyxy:
        # Get bounding box coordinates
        x1, y1, x2, y2 = detection[:4].int().tolist()
        # Create a new track if the vehicle is not already tracked
        if not tracker:
            tracker.append({'id': len(tracker), 'x': x1, 'y': y1, 'w': x2 - x1, 'h': y2 - y1})
        else:
            # Use the Hungarian algorithm to assign the detection to an existing track
            distances = []
            for track in tracker:
                distance = np.sqrt((track['x'] - x1) ** 2 + (track['y'] - y1) ** 2)
                distances.append((distance, track['id']))
            distances.sort()
            assigned_track = distances[0][1]
            tracker[assigned_track]['x'] = x1
            tracker[assigned_track]['y'] = y1
            tracker[assigned_track]['w'] = x2 - x1
            tracker[assigned_track]['h'] = y2 - y1

In [18]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    for detection in results.xyxy:
        # Get bounding box coordinates
        x1, y1, x2, y2 = detection[:4].int().tolist()
        # Draw bounding box and label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {detection[5].int().item()}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [19]:
import time

# Define ROI coordinates for red light
red_light_roi_x1, red_light_roi_y1, red_light_roi_x2, red_light_roi_y2 = 400, 400, 600, 600

# Initialize timer and violation log
timer = {}
viol_log = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    for detection in results.xyxy:
        # Get bounding box coordinates
        x1, y1, x2, y2 = detection[:4].int().tolist()
        # Check if vehicle is in ROI
        if red_light_roi_x1 < x1 < red_light_roi_x2 and red_light_roi_y1 < y1 < red_light_roi_y2:
            # Get vehicle ID and class
            vehicle_id = detection[5].int().item()
            class_name = detection[6].int().item()
            # Check if vehicle has crossed the red light
            if is_red_light(frame):
                # Log violation and save image
                viol_log.append((class_name, vehicle_id, time.time()))
                cropped = frame[y1:y2, x1:x2]
                cv2.imwrite(f'outputs/violations/{class_name}_{vehicle_id}.jpg', cropped)
                print(f'Red light crossing detected: {class_name} {vehicle_id}')

def is_red_light(frame):
    # Implement your red light detection logic here
    # For example, you can use color thresholding to detect the red color
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    return np.any(mask)

In [24]:
import numpy as np

# Initialize tracker
tracker = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    for detection in results.xyxy:
        # Get bounding box coordinates
        x1, y1, x2, y2 = detection[:4].int().tolist()
        # Create a new track if the vehicle is not already tracked
        if not tracker:
            tracker.append({'id': len(tracker), 'x': x1, 'y': y1, 'w': x2 - x1, 'h': y2 - y1})
        else:
            # Use the Hungarian algorithm to assign the detection to an existing track
            distances = []
            for track in tracker:
                distance = np.sqrt((track['x'] - x1) ** 2 + (track['y'] - y1) ** 2)
                distances.append((distance, track['id']))
            distances.sort()
            assigned_track = distances[0][1]
            tracker[assigned_track]['x'] = x1
            tracker[assigned_track]['y'] = y1
            tracker[assigned_track]['w'] = x2 - x1
            tracker[assigned_track]['h'] = y2 - y1

In [20]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    for detection in results.xyxy:
        # Get bounding box coordinates
        x1, y1, x2, y2 = detection[:4].int().tolist()
        # Draw bounding box and label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {detection[5].int().item()}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="78AfLIsP5SIj0tM29qgJ")
project = rf.workspace("popo-rinbd").project("parking-violations-lxf0d")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.64, to fix: `pip install ultralytics==8.0.196`


In [1]:
!nvidia-smi


Wed Jul 24 11:00:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [6]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo mode = checks

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 771, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {MODES}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=<module 'ultralytics.utils.checks' from '/usr/local/lib/python3.10/dist-packages/ultralytics/utils/checks.py'>'. Valid modes are {'benchmark', 'export', 'train', 'predict', 'val', 'track'}.

    Arguments received: ['yolo', 'mode', '=', 'checks']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'segment', 'obb', 'classify', 'pose', 'detect'}
                MODE (required) is one of {'benchmark', 'export', 'train', 'predict', 'val', 'track'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all AR

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="78AfLIsP5SIj0tM29qgJ")
project = rf.workspace("popo-rinbd").project("parking-violations-lxf0d")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.64, to fix: `pip install ultralytics==8.0.196`


In [3]:
!yolo task = detect mode = train model = yolov8m.pt data = {dataset.location}/data.yaml epochs = 20 imgsz = 640

100% 49.7M/49.7M [00:00<00:00, 322MB/s]
Ultralytics YOLOv8.2.64 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/Parking-Violations-2/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

In [4]:
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data = {dataset.location}/data.yaml

Ultralytics YOLOv8.2.64 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/Parking-Violations-2/valid/labels.cache... 65 images, 0 backgrounds, 0 corrupt: 100% 65/65 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 5/5 [00:05<00:00,  1.01s/it]
                   all         65        152      0.852       0.77      0.825      0.674
                   car         65         66      0.957       0.97      0.983      0.904
                 lines         65         86      0.747       0.57      0.666      0.444
Speed: 8.3ms preprocess, 31.1ms inference, 0.0ms loss, 12.1ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [5]:
!yolo task = detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.5 source={dataset.location}/test

Ultralytics YOLOv8.2.64 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 830, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 562, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/predictor.py", line 183, in predict_cli
    for _ in gen:  # sourcery skip: remove-empty-nested-block, noqa
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 35, in generator_context
    response = gen.send(None)
  File "/usr/local/lib/pyt

In [6]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/labels/*.jpg'):
    display(Image(image_path))
    print()